In [1]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm
from tg.grammar_ru.ml.corpus import CorpusReader
from tg.grammar_ru.common import Loc
from pathlib import Path

reader = CorpusReader(Path(Loc.corpus_path/'books.base.zip'))

alg = AntecedentCandidatesAlgorithm(None, 5)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


In [ ]:
from tg.grammar_ru.ml.features import DialogMarkupFeaturizer

dmf = DialogMarkupFeaturizer()
simple_indexes = []
srcs = []
for db in CorpusReader(Path(Loc.corpus_path/'books.base.zip')).get_bundles():
    alg.pronoun_replace_for_db(db)
    dmf.featurize(db)
    no_speech = db.dialog_markup[db.dialog_markup['dialog_type'].isin(['no', 'action'])].index
    db['speechless_src'] = db.src[db.src.index.isin(no_speech)]
    try:
        candidates_df, result_df = alg.run_full(db, neighbour_count=2, neighbour_coeff=2)
        
        merged = candidates_df.merge(result_df, on=['pronoun_word_id','candidate_word_id'], how = 'outer', indicator=True)
        simple_index = merged[['pronoun_word_id','candidate_word_id', 'candidate_distance_x', '_merge']]
        simple_index['is_match'] = simple_index['_merge'] == 'both'
        simple_index = simple_index.rename(columns={'candidate_distance_x': 'candidate_distance'}).drop(columns=['_merge'])
        simple_indexes.append(simple_index)
        srcs.append(db.src)
    except:
        continue

In [15]:
import pandas as pd

simple_index = pd.concat(simple_indexes).reset_index(drop=True)
src = pd.concat(srcs)

In [16]:
simple_index

,pronoun_word_id,candidate_word_id,candidate_distance,is_match
0,7,4,1,True
1,7,5,0,False
2,514887,514884,1,True
3,514887,514885,0,False
4,1903357,1903342,3,False
...,...,...,...,...
101060,22837552,22837513,4,False
101061,22837552,22837524,3,False
101062,22837552,22837537,2,False
101063,22837552,22837545,1,False


In [18]:
import pandas as pd

def create_compare_index(simple_index):
    groups = simple_index.groupby('pronoun_word_id')
    dfs = []
    for pronoun_id,group in groups:
        crossed = group.merge(group, how='cross')
        crossed = crossed[crossed['is_match_x'] != crossed['is_match_y']].reset_index(drop=True)
        crossed = (crossed
                   .rename(columns={"pronoun_word_id_x": "pronoun_word_id", "B": "c"})
                   .drop(columns=['is_match_x', 'pronoun_word_id_y'])
                   .drop(labels=range(len(crossed) // 2), axis=0))
        dfs.append(crossed)
    return pd.concat(dfs, axis=0, ignore_index=True).reset_index(drop=True)

compare_index = create_compare_index(simple_index)
compare_index

,pronoun_word_id,candidate_word_id_x,candidate_distance_x,candidate_word_id_y,candidate_distance_y,is_match_y
0,7,5,0,4,1,True
1,514887,514885,0,514884,1,True
2,1903357,1903351,1,1903348,2,False
3,1903357,1903351,1,1903355,0,False
4,1903357,1903355,0,1903351,1,True
...,...,...,...,...,...,...
59321,22807165,22807163,0,22807158,1,False
59322,22807195,22807193,0,22807182,4,False
59323,22807195,22807193,0,22807183,3,False
59324,22807195,22807193,0,22807188,2,False


In [21]:
from tg.grammar_ru.common import DataBundle

bundle_simple = DataBundle(index=simple_index, src=src)
bundle_compare = DataBundle(index=compare_index, src=src)

In [23]:
from tg.grammar_ru.ml.features import SlovnetFeaturizer, PyMorphyFeaturizer

slvnt = SlovnetFeaturizer()
slvnt.featurize(bundle_simple)
bundle_compare['slovnet'] = bundle_simple['slovnet']

def get_features_for_word_id(series, slovnet_df):
    return (series.to_frame()
            .merge(slovnet_df[['Case', 'relation']], left_on=0, right_index=True, how='left')
            .rename(columns={0: 'word_id'})
            .reset_index(drop=True))

simple_word_ids = pd.concat([simple_index['pronoun_word_id'], simple_index['candidate_word_id']]).drop_duplicates()
compare_word_ids = pd.concat([compare_index['pronoun_word_id'], 
                              compare_index['candidate_word_id_x'],
                              compare_index['candidate_word_id_y']]).drop_duplicates()
bundle_simple['case_relation'] = get_features_for_word_id(simple_word_ids, bundle_simple['slovnet'])
bundle_compare['case_relation'] = get_features_for_word_id(compare_word_ids, bundle_compare['slovnet'])

In [36]:
bundle_simple['case_relation']

,word_id,Case,relation
0,7,Gen,nmod
1,514887,Gen,nmod
2,1903357,NaN,det
3,1903408,Gen,det
4,1903457,NaN,nmod
...,...,...,...
62197,22837513,Gen,nmod
62198,22837524,Acc,obj
62199,22837537,Loc,conj
62200,22837545,Gen,obl


In [37]:
bundle_simple.save(Loc.bundles_path/'antcd/simple')
bundle_simple.save(Loc.bundles_path/'antcd/compare')